In [26]:
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import requests
from datetime import datetime, timedelta
from ebaysdk.finding import Connection as Finding
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import json
import numpy as np

#Loading slack token from .env file. Please provide your own when operating
load_dotenv()

#loading oauth from config.json. please provide your own when operating
with open("config.json", "r") as file:
    config= json.load(file)

# User Input. Change these parameters for the PC part you're looking for
user_input = {
    'product_keywords': "GPU RTX 3060 12GB",
    'condition': "Used",
    'timeframe_days': 50,
    'exclusion_keywords' : ("Broken", "For Parts", "AS IS", "Not Working", "No GPU"),
    'min_price': 100,  # Set a reasonable minimum price
    'max_price': 500  # Set a reasonable maximum price
}
# OAuth token for eBay API
oauth_token = config['OAUTH_TOKEN']

# Function to fetch eBay data using Browse API
def fetch_browse_data(user_input):
    # eBay Browse API endpoint
    api_url = "https://api.ebay.com/buy/browse/v1/item_summary/search"
    
    # Build query string
    query = user_input['product_keywords']
    limit = 50 

    # Construct the final API URL with query and pagination limit
    query_string = f"?q={query}&limit={limit}"
    api_url += query_string

    # Set up the headers with the OAuth token
    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    try:
        # Make the API call
        response = requests.get(api_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if 'itemSummaries' in data:
                filtered_items = []
                exclusion_keywords = user_input['exclusion_keywords']

                # Calculate the timeframe limit based on user input
                timeframe_limit = datetime.now() - timedelta(days=user_input['timeframe_days'])

                for item in data['itemSummaries']:
                    title = item.get('title', "")
                    price = item['price']['value']
                    item_end_time = item.get('itemEndDate')

                    # Exclude items based on keywords and timeframe. But if it's 'buy now', include it anyway
                    if not any(keyword.lower() in title.lower() for keyword in exclusion_keywords):
                        # Convert item_end_time to datetime and filter by timeframe. But if it's 'buy now', include it anyway
                        if item_end_time:
                            item_end_time = datetime.strptime(item_end_time, '%Y-%m-%dT%H:%M:%S.%fZ')
                            if item_end_time >= timeframe_limit:
                                filtered_items.append(item)
                                
                        else:
                            filtered_items.append(item)

                return filtered_items
            else:
                print("No items found in the response.")
                return None
        else:
            print(f"Failed to make API call: {response.status_code}")
            return None

    except Exception as e:
        print(f"API call failed: {str(e)}")
        return None


In [27]:

# Function to calculate average price from the results
def calculate_average_and_median_price(deals):
    if deals is None:
        print("No deals to calculate average price.")
        return None, None

    try:
        # Filter prices within user-specified range
        prices = [float(item['price']['value']) for item in deals if 'price' in item]
        filtered_prices = [price for price in prices if user_input['min_price'] <= price <= user_input['max_price']]

        if filtered_prices:
            average_price = sum(filtered_prices) / len(filtered_prices)
            median_price = np.median(filtered_prices)  # Calculate the median price
            return average_price, median_price
        else:
            print("No prices found in the specified range.")
            return None, None
    except Exception as e:
        print(f"Error calculating prices: {str(e)}")
        return None, None

# Slack notification function
slack_token = os.getenv('SLACK_TOKEN')

client = WebClient(token=slack_token)

def send_slack_notif(deal, average_price):
    deal_url = deal.get('itemWebUrl', 'No URL available')  # Get the URL or return 'No URL available' if not found
    message = (f"💮​ New deal found! {deal['title']} is priced at {deal['price']},"
               f"which is below the average price of {average_price:.2f}. \n"
               f"Check out the deal here: {deal_url}")


    try:
        response = client.chat_postMessage(
            channel='#deals',  # Ensure this is the right channel name
            text=message
        )
        print(f'Notification sent to Slack: {message}')
    except SlackApiError as e:
        print(f"Error sending message to Slack: {e.response['error']}")

# Function to notify of deals below the average price
def notify_of_deals(deals, average_price):
    for deal in deals:
        deal_price = float(deal['price']['value'])

        if deal_price < average_price:
            # Send notification if the price is below the average
            send_slack_notif(deal, average_price)

# Main function
def main():
    # Step 1: Fetch eBay data using Browse API
    deals = fetch_browse_data(user_input)

    # Step 2: Calculate the average and median price
    average_price, median_price = calculate_average_and_median_price(deals)

    if average_price and median_price:
        print(f"\nThe average price for '{user_input['product_keywords']}' is: ${average_price:.2f}")
        print(f"The median price is: ${median_price:.2f}")
        print(f"Number of items analyzed: {len(deals)}")

        # Step 3: Compare prices and send notifications
        notify_of_deals(deals, average_price)
    else:
        print("No deals found matching your criteria.")

# Run the main function
main()



The average price for 'GPU RTX 3060 12GB' is: $300.07
The median price is: $289.97
Number of items analyzed: 46
Notification sent to Slack: 💮​ New deal found! Gigaybyte GV-N3060GAMING OC-12GD REV2.0 GeForce RTX 3060 12GB 192-bit GDDR6 GPU is priced at {'value': '284.99', 'currency': 'USD'},which is below the average price of 300.07. 
Check out the deal here: https://www.ebay.com/itm/185582019265?_skw=GPU+RTX+3060+12GB&hash=item2b358ce2c1:g:l84AAOSwnH5jI7l7&amdata=enc%3AAQAJAAAA8H%2FqIbIc6o8B4%2BvpFzQyE8Vrq3SmUVD7TZQbZKJty1pXr%2F8JpAZJnCNzPC3pF%2FZktgTJQD4%2FPqATH2cBzA3T77KkyliuP0SK12zzh6HBVERc9kF%2BEwnFinOyOB%2Bv%2F9Eg9EfrnUHtJUBLPfX3ovNyN53TzH2is6i%2BqNp4pb9n81%2F3V7TD63wqgT%2BxNI0CJSDcfFtQSK6gc5ME0fN3SDJTYMft0qSf63mAcKagx6XO%2BFBHlI5PSCPLX%2FjnTzUDG2ZWhl3%2FLtUcpVsLX5kL%2BLsZ5I2qrGR3CrvCK%2Fr2TnLt%2BEdLCxOGN51h63qUJ7BRsr9yMA%3D%3D
Notification sent to Slack: 💮​ New deal found! ASUS Strix Nvidia GeForce RTX 3060 OC 12 GB GDDR6 PCIE 4 GPU-Fast Ship🔥 is priced at {'value': '269.99', 'c